In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install plot-metric
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
from plot_metric.functions import BinaryClassification
import scikitplot as skplt
import warnings
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.metrics import make_scorer, recall_score,precision_score, confusion_matrix,classification_report,accuracy_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#!pip install plot-metric

# **Preprocessing**

In [ ]:
data = pd.read_csv("/kaggle/input/banking-dataset-classification/new_train.csv")
data.head()

In [ ]:
topredict = pd.read_csv("/kaggle/input/banking-dataset-classification/new_test.csv")
topredict.head()

In [ ]:
data.shape

In [ ]:
topredict.shape

In [ ]:
data.columns

In [ ]:
topredict.columns

In [ ]:
data=data.drop(["previous", "pdays"], axis = 1)
data

In [ ]:
data.info()

**Descriptive statistics**

In [ ]:
data_num= data.drop(["job", "marital", "education", "default", "housing", "loan", "contact", "month", "day_of_week", "poutcome", "y"], axis=1)
data_num.describe().T

In [ ]:
data_cat= data.drop(["age", "duration", "campaign"], axis=1)
data_cat.describe().T

In [ ]:
#Jobs
plt.figure(figsize=(12, 6))
b = sns.countplot(y ='job', data = data, order=data["job"].value_counts().index)
b.axes.set_title("Jobs",fontsize=22)
b.set_ylabel("Count",fontsize=15)
b.tick_params(labelsize=12)
plt.show()

In [ ]:
#marital
plt.figure(figsize=(12, 6))
b = sns.countplot(x ='marital', data = data, order=data["marital"].value_counts().index)
b.axes.set_title("marital situation",fontsize=22)
b.set_ylabel("Count",fontsize=15)
b.tick_params(labelsize=12)
plt.show()

In [ ]:
#Education level
plt.figure(figsize=(12, 6))
b = sns.countplot(y ='education', data = data, order=data["education"].value_counts().index)
b.axes.set_title("Education level",fontsize=22)
b.set_ylabel("Count",fontsize=15)
b.tick_params(labelsize=12)
plt.show()

In [ ]:
#housing
plt.figure(figsize=(6, 6))
b = sns.countplot(x ='housing', data = data, order=data["housing"].value_counts().index)
b.axes.set_title("Housing situation",fontsize=22)
b.set_ylabel("Count",fontsize=15)
b.tick_params(labelsize=12)
plt.show()

In [ ]:
#outcome of the previous marketing campaign
plt.figure(figsize=(6, 6))
b = sns.countplot(x ='poutcome', data = data, order=data["poutcome"].value_counts().index)
b.axes.set_title("outcome of the previous marketing campaign",fontsize=22)
b.set_ylabel("Count",fontsize=15)
b.tick_params(labelsize=12)
plt.show()

In [ ]:
#has the client subscribed a term deposit?
plt.figure(figsize=(5, 5))
b = sns.countplot(x ='y', data = data, order=data["y"].value_counts().index)
b.axes.set_title("has the client subscribed a term deposit?",fontsize=22)
b.set_ylabel("Count",fontsize=15)
b.tick_params(labelsize=12)
plt.show()

**Missnig values**

In [ ]:
data.isnull().sum()

**Duplicated values**

In [ ]:
data.duplicated().sum()

In [ ]:
#Drops duplicated values
data= data.drop_duplicates()
data_org = data.copy()
len(data)

**Qualitatif variables and discretisation**

In [ ]:
data['job'].value_counts() #discretisation

In [ ]:
## replace rare values with 'Other'
#data= data.replace(to_replace =["student","unknown"], value ="Other") 
#data['job'].value_counts()

In [ ]:
data['marital'].value_counts() #discretisation

In [ ]:
## replace rare values with 'Other'
#data= data.replace(to_replace =["Other"], value ="married") 
#data['marital'].value_counts()

In [ ]:
data['education'].value_counts() #ok

In [ ]:
data['default'].value_counts() #ok

In [ ]:
data['housing'].value_counts() #ok

In [ ]:
data['loan'].value_counts() #ok

In [ ]:
data['contact'].value_counts() #ok

In [ ]:
data['month'].value_counts() #ok

In [ ]:
data['day_of_week'].value_counts() #ok

In [ ]:
data['poutcome'].value_counts() #ok

In [ ]:
data['y'].value_counts() #ok

# **Feature Selection**

In [ ]:
category_vars = list(data.select_dtypes(include='object').columns)
category_vars

In [ ]:
category_vars= category_vars[:-1]
category_vars

**Encoding data**

In [ ]:
## encoding categorecal variables 
for i in category_vars:
    data[i]= data[i].astype('category')
    data[i]= data[i].cat.codes
    data[i]= data[i].astype('float64')
    
data['y'].mask(data['y'] == 'no', 0 , inplace=True)
data['y'].mask(data['y'] == 'yes', 1 , inplace=True)
data['y']= data['y'].astype('float64')

In [ ]:
data

In [ ]:
sns.set_theme(style="white")
corr = data.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, vmin=-1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
data_cp= data.drop('y', axis=1)
corrmat = data_cp.corr()
corrmat = corrmat.abs().unstack() # absolute value of corr coef
corrmat = corrmat.sort_values(ascending=False)
corrmat = corrmat[corrmat >= 0]
corrmat = corrmat[corrmat < 1]
corrmat = pd.DataFrame(corrmat).reset_index()
corrmat.columns = ['feature1', 'feature2', 'correlation']
corrmat.head()

In [ ]:
### spliting data en X et Y
X= data.drop('y', axis=1)
Y= data['y']

In [ ]:
# loop to build a tree, make predictions and get the roc-auc
# for each feature of the train set
roc_values = []
cv = StratifiedKFold(n_splits=20)
for feature in X.columns:
    roc_temp_list = []
    X_= X[feature].copy()
    for train, test in cv.split(X_, Y):
        clf = DecisionTreeClassifier()
        clf.fit(X_.iloc[train].fillna(0).to_frame(), Y.iloc[train])
        y_scored = clf.predict_proba(X_.iloc[test].fillna(0).to_frame())
        roc_temp_list.append(roc_auc_score(Y.iloc[test], y_scored[:, 1]))
    roc_values.append(np.array(roc_temp_list).mean())

In [ ]:
roc_table= pd.DataFrame({'features': X.columns, 'mean roc_auc_score': roc_values})
roc_table=roc_table.reset_index(drop=True)
roc_table=roc_table.sort_values(by=['mean roc_auc_score'], ascending=False)
roc_table

In [ ]:
data_unv= data.drop(['loan', 'housing'], axis=1)

# **Data: Corr**
Using all data

In [ ]:
from imblearn.under_sampling import TomekLinks
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
import warnings
warnings.filterwarnings('ignore')
from imblearn.over_sampling import SMOTE

**Balancing: SMOTE (oversampling)**

data_org is a copy of the preprocessed data after deleting the duplicated values

Data is unbalanced --see graph of y

In [ ]:
data_b = data_org.copy()

In [ ]:
data_b

In [ ]:
#encode the y
data_b['y'].mask(data_b['y'] == 'yes', 1 , inplace=True)
data_b['y'].mask(data_b['y'] == 'no', 0 , inplace=True)
##Spliting to X and Y
X= data_b.drop('y', axis=1)
Y= data_b['y']
Y=Y.astype('int')

In [ ]:
X= pd.get_dummies(X, drop_first= True)

In [ ]:
#balancing the data (SMOTE)
sm =SMOTE()
X_sm, Y_sm = sm.fit_resample(X, Y)

In [ ]:
## return to dataframe structure
X_sm = pd.DataFrame(X_sm, columns=X.columns)

In [ ]:
data_exp1= X_sm
data_exp1['y']= Y_sm

In [ ]:
data_exp1 #SMOTE Balanced data

In [ ]:
#has the client subscribed a term deposit?
plt.figure(figsize=(5, 5))
b = sns.countplot(x ='y', data = data_exp1, order=data_exp1["y"].value_counts().index)
b.axes.set_title("has the client subscribed a term deposit?",fontsize=22)
b.set_ylabel("Count",fontsize=15)
b.tick_params(labelsize=12)
plt.show()

We have a balanced data now.

**Balancing: TomekLinks (undersampling)**

In [ ]:
#balancing the data (TomekLinks)
#tm =TomekLinks()
#X_tk, Y_tk = tm.fit_resample(X, Y)

In [ ]:
#X_tk = pd.DataFrame(X_tk, columns=X.columns)
#data_exp2= X_tk
#data_exp2['y']= Y_tk

In [ ]:
#has the client subscribed a term deposit?
#plt.figure(figsize=(5, 5))
#b = sns.countplot(x ='y', data = data_exp2, order=data_exp2["y"].value_counts().index)
#b.axes.set_title("has the client subscribed a term deposit?",fontsize=22)
#b.set_ylabel("Count",fontsize=15)
#b.tick_params(labelsize=12)
#plt.show()

In [ ]:
#data_exp2['y'].value_counts() #ok

# **Training (SMOTE+CORR)**

**Decision Tree** (Supervised)

In [ ]:
data1=data_exp1.copy()

In [ ]:
X= data1.drop('y', axis=1)
Y= data1['y']

In [ ]:
X_train , X_test , Y_train , Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 123)

In [ ]:
dct = DecisionTreeClassifier()
params={'criterion' : ['gini', 'entropy'],
        'max_depth': [5, 6, 7, 8, 9, 10, 11, 12, 13,14, 15, 16, 17, 18, 19 ,20, None]}

##accuracy
grid_search_acc= GridSearchCV(estimator=dct, param_grid= params, scoring='accuracy',cv=10, n_jobs=-1)
grid_search_acc= grid_search_acc.fit(X_train, Y_train)
y_predict= grid_search_acc.best_estimator_.predict(X_test)

In [ ]:
## evaluating the model
print("----------")
print('best parameters: ', grid_search_acc.best_params_)
print("----------")
print(confusion_matrix(Y_test,y_predict))
print("----------")
print(classification_report(Y_test, y_predict))
print("----------")

In [ ]:
##ROC
y_predict_proba = grid_search_acc.best_estimator_.predict_proba(X_test)[:,1]
#skplt.metrics.plot_roc_curve(Y_test, y_predict_proba)
#plt.show()
# Visualisation with plot_metric
bc = BinaryClassification(Y_test, y_predict_proba, labels=[1, 0])
# Figures
plt.figure(figsize=(7,6))
bc.plot_roc_curve()
plt.title('ROC curve')
plt.show()

In [ ]:
## Recall
rec_avg=make_scorer(recall_score, average='macro')
grid_search_rec= GridSearchCV(estimator=dct, param_grid= params, scoring=rec_avg,cv=10, n_jobs=-1)
grid_search_rec= grid_search_rec.fit(X_train, Y_train)
y_predict= grid_search_rec.best_estimator_.predict(X_test)

In [ ]:
## evaluating the model
print("----------")
print('best parameters: ', grid_search_rec.best_params_)
print("----------")
print(confusion_matrix(Y_test,y_predict))
print("----------")
print(classification_report(Y_test, y_predict))
print("----------")

In [ ]:
## precision
prec_avg=make_scorer(precision_score, average='macro')
grid_search_prec_avg= GridSearchCV(estimator=dct, param_grid= params, scoring=prec_avg,cv=10, n_jobs=-1)
grid_search_prec_avg= grid_search_prec_avg.fit(X_train, Y_train)
y_predict= grid_search_prec_avg.best_estimator_.predict(X_test)

In [ ]:
print("----------")
print('best parameters: ', grid_search_prec_avg.best_params_)
print("----------")
print(confusion_matrix(Y_test,y_predict))
print("----------")
print(classification_report(Y_test, y_predict))
print("----------")

In [ ]:
def plot_grid_search(cv_results, grid_param_1, grid_param_2, name_param_1, name_param_2, titre):
    # Get Test Scores Mean and std for each grid search
    scores_mean = cv_results['mean_test_score']
    scores_mean = np.array(scores_mean).reshape(len(grid_param_2),len(grid_param_1))

    # Plot Grid search scores
    _, ax = plt.subplots(1,1)

    # Param1 is the X-axis, Param 2 is represented as a different curve (color line)
    for idx, val in enumerate(grid_param_2):
        #ax.set_ylim([0.74,0.84])
        ax.plot(grid_param_1[:-1], scores_mean[idx,:-1], '-o', label= name_param_2 + ': ' + str(val))
        ax.plot(19, scores_mean[idx, -1:], '*', label= 'crt' + '=' + str(val) + ' & mx_dpt=None')
        
    ax.set_title(titre, fontsize=18, fontweight='bold')
    ax.set_xlabel(name_param_1, fontsize=14)
    ax.set_ylabel(titre+'[CV Avg Score]', fontsize=14)
    ax.legend(loc="best", fontsize=15)
    ax.grid('on')

In [ ]:
# Calling Method 
plot_grid_search(grid_search_acc.cv_results_, params['max_depth'], params['criterion'],  'max_depth', 'criterion', 'Accuracy')
plot_grid_search(grid_search_rec.cv_results_, params['max_depth'], params['criterion'],  'max_depth', 'criterion', 'Recall')
plot_grid_search(grid_search_prec_avg.cv_results_, params['max_depth'], params['criterion'], 'max_depth', 'criterion', 'Precision')

In [ ]:
##return index for dataframe
def return_index_list(param1, param2):
    list=[]
    for p1 in param1:
        for p2 in param2:
            list.append('['+str(p1)+', '+str(p2)+']')
    return list
index= return_index_list(params['criterion'], params['max_depth'])

In [ ]:
df= pd.DataFrame({  'Accuracy': grid_search_acc.cv_results_['mean_test_score'],
                    'Precision(Avg)': grid_search_prec_avg.cv_results_['mean_test_score'],
                    'Recall(Avg)': grid_search_rec.cv_results_['mean_test_score'],
                     'std_cross_validation': grid_search_acc.cv_results_['std_test_score']},
                 index=index)

In [ ]:
df

**KNN + MaxMin Normalization**

In [ ]:
data2 = data_exp1.copy()

In [ ]:
data2.columns

In [ ]:
cols= ['age','duration', 'campaign']

In [ ]:
for col in cols:
    data2[col] = (data2[col] - data2[col].min())/(data2[col].max() - data2[col].min())

In [ ]:
import warnings
from plot_metric.functions import BinaryClassification
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
from sklearn.model_selection import GridSearchCV, train_test_split
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import make_scorer, recall_score,precision_score, confusion_matrix,classification_report,accuracy_score

In [ ]:
data_k= data2.copy()
X= data_k.drop('y', axis=1)
Y= data_k['y']

In [ ]:
X_train , X_test , Y_train , Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 123)

In [ ]:
np.unique(Y_train, return_counts=True)

In [ ]:
knn = KNeighborsClassifier()
params={'n_neighbors' : [2, 3, 4, 5, 6, 7, 8],
        'metric': ['euclidean','manhattan']}

##accuracy
grid_search_acc_k= GridSearchCV(estimator=knn, param_grid= params, scoring='accuracy',cv=10, n_jobs=-1)
grid_search_acc_k= grid_search_acc_k.fit(X_train, Y_train)
y_predict= grid_search_acc_k.best_estimator_.predict(X_test)

In [ ]:
## evaluating the model
print("----------")
print('best parameters: ', grid_search_acc_k.best_params_)
print("----------")
print(confusion_matrix(Y_test,y_predict))
print("----------")
print(classification_report(Y_test, y_predict))
print("----------")


##ROC
y_predict_proba_k = grid_search_acc_k.best_estimator_.predict_proba(X_test)[:,1]

# Visualisation with plot_metric
bc_k = BinaryClassification(Y_test, y_predict_proba_k, labels=[1, 0])
# Figures
plt.figure(figsize=(7,6))
bc_k.plot_roc_curve()
plt.title('ROC curve')
plt.show()

In [ ]:
## Recall
rec_avg_k=make_scorer(recall_score, average='macro')
grid_search_rec_k= GridSearchCV(estimator=knn, param_grid= params, scoring=rec_avg,cv=10, n_jobs=-1)
grid_search_rec_k= grid_search_rec_k.fit(X_train, Y_train)
y_predict= grid_search_rec_k.best_estimator_.predict(X_test)

In [ ]:
## evaluating the model
print("----------")
print('best parameters: ', grid_search_rec_k.best_params_)
print("----------")
print(confusion_matrix(Y_test,y_predict))
print("----------")
print(classification_report(Y_test, y_predict))
print("----------")

In [ ]:
## precision
prec_avg_k=make_scorer(precision_score, average='macro')
grid_search_prec_avg_k= GridSearchCV(estimator=knn, param_grid= params, scoring=prec_avg,cv=10, n_jobs=-1)
grid_search_prec_avg_k= grid_search_prec_avg_k.fit(X_train, Y_train)
y_predict= grid_search_prec_avg_k.best_estimator_.predict(X_test)

In [ ]:
## evaluating the model
print("----------")
print('best parameters: ', grid_search_prec_avg_k.best_params_)
print("----------")
print(confusion_matrix(Y_test,y_predict))
print("----------")
print(classification_report(Y_test, y_predict))
print("----------")

In [ ]:
# Calling Method 
plot_grid_search(grid_search_acc_k.cv_results_, params['n_neighbors'], params['metric'], 'N neighbors', 'Metric', 'Accuracy')
plot_grid_search(grid_search_rec_k.cv_results_, params['n_neighbors'], params['metric'], 'N neighbors', 'Metric', 'Recall')
plot_grid_search(grid_search_prec_avg_k.cv_results_, params['n_neighbors'], params['metric'], 'N neighbors', 'Metric', 'Precision')

In [ ]:
def return_index_list(param1, param2):
    list=[]
    for p1 in param1:
        for p2 in param2:
            list.append('['+str(p1)+', '+str(p2)+']')
    return list
index= return_index_list(params['metric'], params['n_neighbors'])

In [ ]:

df_knn= pd.DataFrame({  'Accuracy': grid_search_acc_k.cv_results_['mean_test_score'],
                    'Precision(Avg)': grid_search_prec_avg_k.cv_results_['mean_test_score'],
                    'Recall(Avg)': grid_search_rec_k.cv_results_['mean_test_score'],
                     'std_cross_validation': grid_search_acc_k.cv_results_['std_test_score']},
                 index=index)